# Local Project Migration to the DevCloud

This tutorial explains the differences between running your IoT application in your local environment and the DevCloud, and how to migrate a working application in your local environment to the DevCloud. Because everyone's application has unique requirements, it is intended as a guide and not the definitive reference for doing so.

For the purpose of this tutorial, we will use the [Image Segmentation Python Demo](https://github.com/opencv/open_model_zoo/tree/master/demos/python_demos/segmentation_demo) from the Open Model Zoo as our sample to migrate to the DevCloud.

## OpenVINO version check:
You are currently using the latest development version of Intel® Distribution of OpenVINO™ Toolkit. Alternatively, you can open a version of this notebook for the Intel® Distribution of OpenVINO™ Toolkit LTS version by running the cell below and following the link it generates.

In [ ]:
from qarpo import displayMultiversionURL
import os
displayMultiversionURL(os.path.abspath(""), "Local%20Project%20Migration%20to%20the%20DevCloud.ipynb", "openvino-dev-latest", ['openvino-lts'])

## Understanding what the DevCloud does/does not do
The Intel DevCloud for the Edge is a useful tool for quickly prototyping, developing and optimizing a new application or importing an existing application and then analyzing the application performance across multiple hardware environments. Since the hardware is in the cloud, it is not necessary to purchase specific hardware on which to run your application until you are closer to deploying it. The DevCloud environment uses consistent interfaces to the AI tools to facilitate being able to deploy on multiple types of hardware. An overview of the DevCloud for the Edge can be found at [here](https://devcloud.intel.com/edge/get_started/devcloud/)

### How components of the DevCloud translate to local environments

From the illustration below some components of the DevCloud are not present or different in a local execution environment. The circled numbers refer to the DevCloud graphic below. 
- The HTTP browser❶ access to Development Servers❷ is typically used to replace direct terminal access in your local environment
- Your local development machine replaces the Development Servers❷ and the Jupyter Notebook. 
- The Storage Server, accessed as `/data` within the Development Server environment, likely does not exist in your local environment. If you have large datasets, you may want to upload them to `/data` when migrating to the DevCloud.
- The Job Queue❸ is the means within the DevCloud to share access to limited hardware resources❹ in the cloud. Your local deployment environment will typically have only one hardware resource and dedicated access. In order to take advantage of shared hardware environments, you local application will need to be adapted to use the queuing system to access that hardware.
- Instead of viewing output directly in your local environment, your application will need to be adapted to store/view the Inference Video/Image Output❺ and the Results❻ via an HTTP browser.

<img src="https://devcloud.intel.com/edge/static/images/svg/Edge-howitworks_PN3.svg" />

## Prerequisites and assumptions in the local environment
Although this tutorial references a specific demo from the Open Model Zoo samples, the concepts and the steps will also be necessary for your own project migration to the DevCloud

For this tutorial we will be migrating from a local server (using an Intel® Core™ i7-8700 CPU @ 3.20GHz with 16GB of memory) running an instance of Ubuntu 18.04 with Intel® Distribution of OpenVINO™ toolkit 2019 R3 and its dependencies installed. This should include Open Model Zoo tools. The target environment will be the CPU of this machine.

### Installed matching version of OpenVINO and its prerequisites

<br><div class=note><i><b>Note: </b>The Intel® Distribution of OpenVINO™ Toolkit 2019 R3 version is what is currently used in the DevCloud. As the DevCloud platform is updated, the version installed in your local environment should match. Mismatched versions might lead to execution failures and unnecessary troubleshooting.</i></div>

To ensure a successful migration, validate that your installed local Intel® Distribution of OpenVINO™ toolkit version matches the version you will be using in the DevCloud. Run the command below to see which version of OpenVINO has been linked to `/opt/intel/openvino/`. You can run the same command (less the initial "!") in your local environment. If they do not match, download and install the matching version of Intel® Distribution of OpenVINO™ toolkit in your local environment.

In [ ]:
!awk '/OpenVINO/{gsub("for.*$","");print;exit}' /opt/intel/openvino/licensing/OpenVINOsupport.txt

### Make the application run in the local environment
If you are moving an application to the DevCloud, we assume that the application is already working in your local environment. The steps that follow in this section are primarily to assemble a local sample application that will be migrated to the DevCloud. Secondarily the steps provide a reference to similar steps you will have done to achieve a working application in your local environment. If your application is working in your local environment, the steps below are for reference only and do not need to be executed.

We will first demonstrate running the [Image Segmentation Python Demo](https://github.com/opencv/open_model_zoo/tree/master/demos/python_demos/segmentation_demo) locally. The following instructions are intended to be run in your local environment.

The Image Segmentation Demo primarily consists of a single Python application code file, two sets of inferencing model files from the Model Zoo, and a single sample input image file for testing. We will create a project directory containing the application code. The model files will be placed in a `model` subdirectory, and the input image will be placed in a `data` subdirectory,

#### Verify the installation of OpenVINO™ and tools
Run the following in a user terminal window on Ubuntu to verify the required tools are in place.

```bash
# define a few well known directories to make it easier to refer to them
export OPENVINO_DIR=/opt/intel/openvino
export MODEL_ZOO_DIR=$OPENVINO_DIR/deployment_tools/open_model_zoo
export ZOO_TOOLS_DIR=$MODEL_ZOO_DIR/tools/
export ZOO_DEMOS_DIR=$MODEL_ZOO_DIR/demos/python_demos
export CPU_EXTENSION_LIB=~/inference_engine_samples_build/intel64/Release/lib/libcpu_extension.so

# Verify that the Intel(R) Distribution of OpenVINO(TM) toolkit is installed
# output should be "Intel(R) OpenVINO(TM) toolkit 2019 R3" or an alternate compatible version
awk '/OpenVINO/{gsub("for.*$","");print;exit}' \
   $OPENVINO_DIR/licensing/OpenVINOsupport.txt

# Verify that Open Model Zoo is installed
# Output should be "Model Zoo Downloader found"
if test -f "$ZOO_TOOLS_DIR/downloader/downloader.py"; then echo "Model Zoo Downloader found"; fi

# Verify demo exists
# Output should be "Model Zoo demo found"
[[ -f "$ZOO_DEMOS_DIR/segmentation_demo/segmentation_demo.py" ]] && echo "Model Zoo demo found"
```

If you do not find Model Zoo installed, you may need to use git to clone using a command similar to below:

```bash
# Clone the Model Zoo if it does not exist
if [[ ! -d "$MODEL_ZOO_DIR" ]] then
    git clone https://github.com/opencv/open_model_zoo.git $MODEL_ZOO_DIR
fi
```

The CPU extension library is normally created while running the demo steps that verify the installation of the Intel® Distribution of OpenVINO™ toolkit. Run the following to verify that the required CPU extension library has been compiled and is available:

```bash
[[ -f "$CPU_EXTENSION_LIB" ]] && echo "$CPU_EXTENSION_LIB found."
```

If the CPU extension library file could not be found, you will need to execute the `build_samples.sh` and then re-run the above to verify it was built. (There is no published method for building just the CPU Extension Library without the samples at this time.)

```bash
# Build the libcpu_extension.so using the samples build script
cd /opt/intel/openvino/deployment_tools/inference_engine/samples
sudo ./build_samples.sh
[[ -f "$CPU_EXTENSION_LIB" ]] && echo "$CPU_EXTENSION_LIB found."
```

#### Download optimized models  from Model Zoo
For this tutorial we will use two different models that have already been optimized through the Model Optimizer. We will use the Model Zoo downloader utility to get them and place them in a `models` directory.

```bash
# First create a directory for this tutorial project and the models and go there
mkdir -p ~/image_segmentation/models && cd $_

# Download the two models
$ZOO_TOOLS_DIR/downloader/downloader.py --name "road-seg*"
$ZOO_TOOLS_DIR/downloader/downloader.py --name "semantic-seg*"
```

For your application, you will want to have the required optimized model files located in your project directory for when you upload the project. If your models have not yet been optimized for use in the Intel® Distribution of OpenVINO™ toolkit, refer to the [Model Optimizer Developer Guide](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) for accomplishing this task.

#### Sample Data available
Next we locate our data. For this tutorial, we will use an existing image file from the [Intel® Distribution of OpenVINO™ Toolkit Car Detection Tutorial](https://github.com/intel-iot-devkit/inference-tutorials-generic/tree/openvino_toolkit_2019_r1_0/car_detection_tutorial).

```bash
# get a data sample into a subdirectory of the project to use for testing
mkdir -p ~/image_segmentation/data && cd $_
wget "https://github.com/intel-iot-devkit/inference-tutorials-generic/raw/openvino_toolkit_2019_r1_0/car_detection_tutorial/data/car_1.bmp"
```

The sample file contains this image:
<img src="resources/car_1.bmp" />

#### Locate and copy the application code
For this tutorial we simply copy the Python code from the Model Zoo demos to our project directory for easy access. Instructions and additional information for all of the demos in Model Zoo are found [here](https://github.com/opencv/open_model_zoo/blob/master/demos/README.md)

```bash
# Copy the application source code from the Model Zoo demos to the local project directory
cp $ZOO_DEMOS_DIR/segmentation_demo/* ~/image_segmentation/
ls -R1 ~/image_segmentation/
```

The local project directory should now look like the following:
<img src="resources/image_segmentation_tree.png" />

#### Execute the local application - road segmentation model
Next we will run the image segmentation demo using both models to verify that it runs locally. We will specify the input file, the model, and the location of the CPU extension library, because we will use the default CPU device. 

<br><div class=note><i><b>Note: </b>This tutorial assumes that Python version 3 is the configured default version in the local environment. To be explicit, we will use the `python3` to invoke Python because the demo requires version 3.</i></div>

First we will run the code with the road segementation model.

```bash
cd ~/image_segmentation

# See what input parameters are required
python3 segmentation_demo.py --help

# Run with the required parameters using the road_segmentation model
python3 segmentation_demo.py \
   -i data/car_1.bmp \
   -m models/intel/road-segmentation-adas-0001/FP32/road-segmentation-adas-0001.xml \
   -l $CPU_EXTENSION_LIB
```

The output execution log will show:
<img src="resources/image_segmentation_road_output.png" />

View the output image to verify that the foreground area of the road is colorized.

```bash
eog out_0.bmp
```
The output image will appear as:
<img src="resources/image_segmentation_road_out_0.bmp" />

#### Execute the local application - semantic segmentation model
Now we will run the demo using the semantic segmentation model.

```bash
# Run with the required parameters using the semantic_segmentation model
python3 segmentation_demo.py \
   -i data/car_1.bmp \
   -m models/intel/semantic-segmentation-adas-0001/FP32/semantic-segmentation-adas-0001.xml \
   -l $CPU_EXTENSION_LIB
```

The output execution log will show:
<img src="resources/image_segmentation_semantic_output.png" />

View the output image to verify that the cars are colorized the same color and the road and background show other colors.

```bash
eog out_0.bmp
```
The output image will appear as:
<img src="resources/image_segmentation_semantic_out_0.bmp" />

## Prerequisites in the DevCloud
The Intel® Distribution of OpenVINO™ toolkit, including the Open Model Zoo tools, are already loaded in your DevCloud instance, so there are no explicit actions required to make these tools available.

## Uploading required files
The DevCloud Jupyter Notebook environment allows uploading a single file at a time. For this tutorial, we will bundle our application into an archive file, upload the archive file, and extract the archived files into a project directory.

### Create archive(s)
Because we put all relevant models, data, and application files in the directories and subdirectories of `~/image_segmentation` on our local machine, we simply need to create gzipped tar archive of that directory.

```bash
# Create a tgz
cd ~ && tar -cvzf image_segmentation.tgz image_segmentation
```

<br><div class=warn><b>Warning: </b>Jupyter uploads and downloads are typically limited to 100MB due to limitations built into the dependent Tornado code. If your zip file is larger than 100MB, you will need to split it into smaller files and upload each of them, using the following commands instead of the above tar command:

```bash
cd ~
tar -czvf - image_segmentation | split --bytes=100MB - ~/image_segmentation.tgz.part-
```

This creates multiple files with the prefix 'image_segmentation.tgz.part-' followed by 'aa', 'ab', etc. After uploading these to the DevCloud, you can re-combine and unpack them in a terminal with:

```bash
cat image_segmentation.tgz.part-* | tar -xvzf -
```
</i></div>

If you have other sources for local files, you will need to modify the tar commands above to create archive(s) of your files for upload.

### Upload archive(s) to the DevCloud
We assume you have user credentials to the DevCloud and have launched a browser and browsed to [https://devcloud.intel.com/edge/advanced/connect_and_create/](https://devcloud.intel.com/edge/advanced/connect_and_create/), clicked on the `My Files` button, and logged in.

1. Upload your archive. Click the `Upload` button and navigate in the pop-up file dialog to the archive created above on your local machine, `image_segmentation.tgz`, select it and press the `Open` button. You will likely get a confirmation dialog about the file size being large asking whether you want to continue. Click `Yes`. (See note above if your file is larger than 100MB.)
<img src="resources/screenshot-upload-1.png" />

You will then see the following screen where you need to again click the indicated `Upload` button to continue the upload.
<img src="resources/screenshot-upload-2.png" />

The `Upload` button will change to `Uploading` and then an increasing percentage of the upload completion. Wait for the upload(s) to complete before proceeding to the next step.

2. Launch a Linux terminal by clicking `New` and then `Terminal` to prepare for the next step:
<img src="https://devcloud.intel.com/edge/static/images/png/edge-jupyter-new-terminal.png" />

## Extract the archive
Next we will extract the files from the archive we uploaded into our home directory on the DevCloud. In the terminal window on the DevCloud run the following:

```bash
# go to our home directory
cd ~ 

# extract the archive
tar -xvzf image_segmentation.tgz
```
<br><div class=note><i><b>Note: </b>If you previously split your archive into multiple files to maintain file sizes of less than 100MB, you would now use the command below to re-combine and unpack them:

```bash
cat image_segmentation.tgz.part-* | tar -xvzf -
```
</i></div>


Our project is now in the `~/image_segmentation` directory.

## Preparing for execution in the DevCloud
There are two fundamental methods for creating and executing a project in the DevCloud. The first is to create a Jupyter Notebook and put Python code snippets and descriptions into the notebook to be executed incrementally as a training exercise. The second is to create (or import) the Python (or C++) code into your DevCloud workspace and execute it from a terminal similar to what you would have done in a local development environment. We will mostly follow the second method and make only the changes necessary to more effectively use the DevCloud.

### DevCloud directory changes
If you were going to run your project as a Jupyter Notebook, you would want to create your project directory under `~/My-Notebooks` so that it is easily accessible from the Notebook. For our purposes in this tutorial, we have extracted the archive into `~/image_segmentation`. 

Similar to what we did in the local environment, we will define some local environment variables for common directories to make referencing them easier and more clear. In the DevCloud terminal window run the following:

```bash
# define a few well known directories to make it easier to refer to them
export OPENVINO_DIR=/opt/intel/openvino
export MODEL_ZOO_DIR=$OPENVINO_DIR/deployment_tools/open_model_zoo
export ZOO_TOOLS_DIR=$MODEL_ZOO_DIR/tools/
export ZOO_DEMOS_DIR=$MODEL_ZOO_DIR/demos/python_demos
export CPU_EXTENSION_LIB=~/inference_engine_samples_build/intel64/Release/lib/libcpu_extension.so

# also add some definitions for the project directory
export APP_DIR=~/image_segmentation
export DATA_DIR=$APP_DIR/data
export MODEL_DIR=$APP_DIR/models
```

Run the following to verify that the required CPU extension library has been compiled and is available in the DevCloud

```bash
[[ -f "$CPU_EXTENSION_LIB" ]] && echo "$CPU_EXTENSION_LIB found."
```

If the CPU Extension library file could not be found, you will need to execute the `build_samples.sh` and then re-run the above to verify it was built. (There is no published method for building just the CPU Extension Library without the demos at this time.)

```bash
# Build the libcpu_extension.so using the samples build script
cd /opt/intel/openvino/deployment_tools/inference_engine/samples
./build_samples.sh
[[ -f "$CPU_EXTENSION_LIB" ]] && echo "$CPU_EXTENSION_LIB found."
```

### Source changes

At this point, with terminal access we might think we would be able to execute the uploaded application in the same manner as was done in the local environment. However, the DevCloud [login node (terminal) environment is not configured to be a target CPU compute device](https://devcloud-docs.readthedocs.io/en/latest/connect.html#if-you-connect-with-the-terminal); the [login node (vs. compute node)](https://devcloud-docs.readthedocs.io/en/latest/faq.html#login-node-versus-compute-node) does not have the memory and CPU resources to run inference. To run code on shared compute device(s), you must submit a job to a queue to get access to the compute nodes with edge inference devices. The following section steps through the types of changes to the source code required to enable running in a queued job environment with shared compute devices. 

To accomplish the source changes, return to your DevCloud Jupyter file management tab, click on the `image_segmentation` directory we had previously created, then click on `segmentation_demo.py` to open an editor on the code.

#### Adding job ID to log and output file names
When a job is submitted to a queue, a unique job ID is generated. When jobs are completed, their output is visible in the project directory. Therefore, it is necessary to name output files using the job ID to distinguish between possible multiple job runs. We will accomplish this by adding command line parameters to our application for specifying the output path and output filename and making code changes to include the job ID as part of those filenames.

When a job is run on a compute device, it may run in a different base directory than what is defined for the project. Therefore any subdirectories and files should be referenced as relative from the base and not use absolute paths.

The default Python logger module sends output to the console. Although the console of each job can be monitored, it is often desirable to record the output to a specific directory and file. Therefore, the log file names should also include the job ID. We will use the output path parameter and add a log filename to the command line parameters and change the code for setting up logging.

The new command line parameters will default to reasonable values. if not supplied, and may be used more explicitly in the later creation of job files.

In the editor window for `segmentation_demo.py` scroll to the `build_argparser()` function and just before the `return parser` (line 69) insert the following new lines to define these new arguments:


<img src="resources/build_argparser_line_69.png" />

```python
    args.add_argument("-p", "--out_dir", 
                      help="Optional. The path where result files and logs will be stored",
                      required=False, default="./results", type=str)
    args.add_argument("-o", "--out_prefix", 
                      help="Optional. The file name prefix in the output_directory where results will be stored", 
                      required=False, default="out_", type=str)
    args.add_argument("-g", "--log_prefix", 
                      help="Optional. The file name prefix in the output directory for log files",
                      required=False, default="log_", type=str)
```

Next we change the first two lines of main() (highlighted lines 82-83).

<img src="resources/segmentation_demo_py_lines_82-83.png" />

Change these two lines to the following:

```python
    job_id = os.getenv("PBS_JOBID")
    args = build_argparser().parse_args()
    
    # Ensure the out_dir exists
    results_dir = os.path.join(os.path.dirname(__file__), args.out_dir)
    if not os.path.exists( results_dir ):
        os.mkdirs( results_dir )

    # Set up logging to a file
    logpath = os.path.join(os.path.dirname(__file__), 
                           "{}/{}_{}.log".format(args.out_dir, args.log_prefix, job_id))
    log.basicConfig(level=log.INFO,
                    format="%(asctime)s %(name)-12s %(levelname)-8s %(message)s",
                    filename=logpath,
                    filemode="w" )
    
    # Setup additional logging to console
    console = log.StreamHandler()
    console.setLevel(log.INFO)
    formatter = log.Formatter("[ %(levelname)s ] %(message)s")
    console.setFormatter(formatter)
    log.getLogger("").addHandler(console)

```

The first line above retrieves the environment variable containing the job ID. This is supplied by the queue manager when a job is submitted. Then we build the command line parser and parse the options. Next we create the output results directory.

The next section sets up basic logging to a file using the results directory and a log file named in the format `<log_prefix>_<job_id>.log`. The log lines will include a time, the username, the log severity level, and the message.

The final section above adds an additional logging handler to copy the logs to the console in a more simplified style. This section is optional and depends on whether you prefer logging to the console, a file, or both.

If your application was only writing progress to the console using print statements, you would need to modify it to use the logging.py module interfaces.

The final section that we need to modify in this example is what is now line ~171 (highlighted below):
<img src="resources/segmentation_demo_py_line_171.png" />

Change this line to be:

```python
        out_img = os.path.join(os.path.dirname(__file__), 
                               "{}/{}_{}_{}.bmp".format(args.out_dir, args.out_prefix, job_id, batch))
```

This change will cause the output .bmp image to be written to our designated results directory in the format `<prefix>_<job_id>_<batch_number>.bmp` (`<batch_number>` for if multiple batches are indicated by the loaded inference engine network).


### Creating a job file
Job files, which are specialized bash shell scripts, are used to submit application workload(s) to the shared edge compute nodes in the DevCloud. Job files are a wrapper around the Python code execution to be executed on the selected edge compute node. To pass specific variables to our Python code, we will create convenient command line options for the job file to use in creating the command line options for the Python code. The migration of your specific application may require options not shown here, but you can use what follows as a basic template for a new job file. 

For the image_segmentation migration, the following options to the job file will be supported:
* -d    the edge compute device name to use ("CPU", "GPU", "MYRIAD", or "HDDL")
* -i    the input path/filename
* -r    the base directory path for results and logs
* -m    the base directory path for models (assumes directory structure of base/model_name/FPxx/model_name.xml|bin
* -f    the floating point model to use on the device ("FP16" or "FP32)
* -x    the model name

Create an `image_segmentation_job.sh` file using either `vi image_segmentation_job.sh` from the terminal windows or by returning to the Jupyter file manager, navigating to the `image_segmentation` directory, clicking on the `New -> Text File` button, then clicking on the `untitled.txt` name at the top of the new file and renaming it to `image_segmentation_job.sh`.

Add the contents of the code blocks below to the file. An explanation of the code follows each code block.

```bash
ME=`basename $0`

# The default path for the job is your home directory, so we change directory to where the files are on the compute device.
cd $PBS_O_WORKDIR
```

The shell variable `ME` is set to the name of this script to be used for progress output from the job file. 

When the job is launched, the current working directory is on your login device. However, the job actually runs on the compute device. The queuing system provides a list of environment variables that can be used on the launched compute device, and the `$PBS_O_WORKDIR` is the working directory on that device where your files are copied when it is launched.

```bash
# Set default values of command line options
USAGE="USAGE: $ME [-d DEVICE] [-i INPUT_FILE] [-r RESULT_DIR] [-m MODEL_BASE_DIR] [-f (FP16|FP32)] [-x MODEL_NAME] [-l CPU_EXTENSION_LIB]"
DEVICE=CPU
INPUT_FILE=
RESULTS_BASE=results
MODEL_BASE=models/intel
FP_MODEL=FP32
MODEL=semantic-segmentation-adas-0001
CPU_EXTENSION_LIB=/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so

#Parse command line options
while getopts 'd:i:r:m:f:x:?' OPTION; do
    case "$OPTION" in
    d)
        DEVICE=$OPTARG
      ;;
    i)
        INPUT_FILE=$OPTARG
      ;;
    r)
        RESULTS_BASE=$OPTARG
      ;;
    m)
        MODEL_BASE=$OPTARG
      ;;
    f)
        FP_MODEL=$OPTARG
      ;;
    x)
        MODEL=$OPTARG
      ;;
    c)
        CPU_EXTENSION_LIB=$OPTARG
      ;;
    ?)
        echo $USAGE
        exit 0
      ;;
    /?)
        echo -e "Invalid argument: $OPTARG\n$USAGE" && exit 1
      ;;
    :)
        echo -e "Missing argument: $OPTARG\n$USAGE" && exit 1
      ;;
    esac  
done

# Print settings being used
echo "$ME is using device $DEVICE"
echo "$ME is using input file $INPUT_FILE"
echo "$ME is using $MODEL_BASE for the model base directory"
echo "$ME is using floating point model $FP_MODEL"
echo "$ME is using the $MODEL model"
echo "$ME is using $CPU_EXTENSION_LIB for the CPU extension library"
echo "$ME is using $RESULTS_BASE for the base directory for storing output and logs"

MODEL_XML="${MODEL_BASE}/${MODEL}/${FP_MODEL}/${MODEL}.xml"

# Ensure that the results directory exists
mkdir -p $RESULTS_BASE
```

The code block above defines default values and parses the command line options for the job script using the built-in bash function getopts. It then combines the model path arguments to generate the full path of the .xml model file and ensures that we have an output directory.


```bash
# Run the image_segmentation Python code
APP_PATH=$PBS_O_WORKDIR
python3 segmentation_demo.py \
   -d $DEVICE \
   -i $INPUT_FILE \
   -m $MODEL_XML \
   -l $CPU_EXTENSION_LIB \
   -p $RESULTS_BASE

```
The last code block above executes the Python code passing the appropriate parameters.

See [Next Steps and Links to further reading](#Next-Steps-and-Links-to-further-reading) for more information on submitting and managing jobs.

## Execute the DevCloud application

Now that we have a job file and the underlying application, we can submit jobs to compute nodes.

To determine what nodes are available execute the following command in your DevCloud terminal:

```bash
pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c
```
The output should similar to:

<img src="resources/pbsnodes_output.png" />

The number at the left indicates how many nodes of that type are available. The remaining string identifies properties of that node type. You use these properties to select a node when you submit a job. The properties to the left are more general and become more specific as you move to the right.


### Submitting job to run on Intel® CPU
In the section below are the terminal commands to submit a job to an <a href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI Tank 870-Q170</a> edge node with an <a  href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500TE</a>. The inference workload will run on the CPU.
    
```bash
#Submit job to the queue
JOB_ID_CORE=`qsub image_segmentation_job.sh \
             -l nodes=1:idc001skl \
             -F " -i data/car_1.bmp " \
             -N image_segmentation_core`
echo $JOB_ID_CORE

#Progress indicators
QSTAT_OUT=; \
while [[ ! $QSTAT_OUT =~ Unknown ]]; \
do \
    QSTAT_OUT=$(qstat $JOB_ID_CORE 2>&1); \
    echo $QSTAT_OUT; \
    sleep 2s; \
done
```
The first command runs `qsub` (job submission) with the following options:
* -l nodes=1:idc001skl - Specifies the required properties of the node(s) to use ("node=" followed by ':' separated list of properties as reported by the `pbsnodes | grep compnode` command)
* -F " -i data/car_1.bmp " - Specifies the command line arguments when running the job file `image_segmentation_job.sh`
* -N image_segmentation_core - Specifies the name of the job and becomes base for output file names
**Note** that because we created reasonable defaults for many of the command line options in the job file, we do not need to specify all options here as part of the -F string of options.

The first command also captures the job ID and server name from the `qsub` output into the variable `JOB_ID_CORE`. We then run `qstat` (job status) on the value of `JOB_ID_CORE` every two seconds until the stderr output contains "Unknown" indicating that the job is no longer running.  The output from running the commands will appear similar to the following:

![](./resources/running_cpu_job_output.png)

After a successful completion of the job, the project directory will contain several new files similar to:
* **image_segmentation_core.e2477** -- stdout from the Python application code
* **image_segmentation_core.o2477** -- stdout from the job script
* **results/log_2477.v-qsvr-1.devcloud-edge.log** -- the log file output of the job
* **results/out_2477.v-qsvr-1.devcloud-edge_0.bmp** -- the resulting segmented image file

In the components of the file names:
* The "2477" is the job ID (instance ID)
* The "v-qsvr-1.devcloud-edge" is the server name on which the job was run
* The "image_segmentation_core" is from the `-N image_segmentation_core` option in the qsub command which provided a name for the job.


### Viewing output

The DevCloud provides a set of Python functions in the [demoutils.py module](https://github.com/intel-iot-devkit/iot-devcloud/blob/master/demoTools/demoutils.py) to make it easier to display qstat output in a notebook and displaying progress indicators. It also includes functions to display images and video. You are encouraged to view this code and incorporate it into your application as desired when running on the DevCloud. 

### Validating performance

Now that you have seen a basic job file, you can explore running the application on other compute nodes and devices by modifying the node requested and the device parameters. You may have to instrument your application to provide better timing for comparing across nodes. You may also want to explore the [Benchmark Python Tool](https://docs.openvinotoolkit.org/latest/_inference_engine_tools_benchmark_tool_README.html) in the OpenVINO toolkit.


## Next Steps and Links to further reading

* Intel documentation on [Basic Job Submission](https://devcloud-docs.readthedocs.io/en/latest/job.html)
* Intel documentation on [Running Multiple Jobs](https://devcloud-docs.readthedocs.io/en/latest/multi_job.html)
* For more information on the job queue, job parameters, managing the queue, and selecting compute nodes refer to [Advanced Queue Management](https://devcloud.intel.com/datacenter/learn/advanced-queue)
* [Intel DevCloud FAQ](https://devcloud-docs.readthedocs.io/en/latest/faq.html)
* For additional information about the job queuing system used on DevCloud, read about [Adaptive Computing's TORQUE Resource Manager](https://support.adaptivecomputing.com/support/documentation-index/torque-resource-manager-documentation/). Of particular interest is [Chapter 3: Submitting and Managing Jobs](http://docs.adaptivecomputing.com/torque/6-1-2/adminGuide/torque.htm#topics/torque/2-jobs/submittingManagingJobs.htm%3FTocPath%3DChapter%25203%253A%2520Submitting%2520and%2520Managing%2520Jobs%7C_____0)
* [More information about using the Python argparse module](https://docs.python.org/3/library/argparse.html) for parsing command line arguments 
* [More information about the Python logging module](https://docs.python.org/3/library/logging.html#module-logging)

### Optimization
Generally you use the DevCloud environment for optimizing your application for use with the available target device(s) before you port it back to your local environment. Rather than replicating multiple ideas for further optimization of your application, for considering latency versus batch size, and for other adjustments, the following links provide more information.

#### Intel Links

- [OpenVINO Model Optimizer Developer Guide](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
- [OpenVINO Optimization Guide](https://docs.openvinotoolkit.org/latest/_docs_optimization_guide_dldt_optimization_guide.html) (this is a sub-link of the Model Optimizer Developer Guide)
- [Model Optimization Techniques](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_prepare_model_Model_Optimization_Techniques.html) (this is a sub-link of the Model Optimizer Developer Guide)
(and further links accessible from these pages)
- [CPU Inference Performance Boost with “Throughput” Mode in the Intel® Distribution of OpenVINO™ Toolkit](https://www.intel.ai/cpu-inference-performance-boost-openvino/#gs.uqlebj)

#### Non-Intel Links
- [Optimizing neural networks for production with Intel’s OpenVINO](https://hackernoon.com/optimizing-neural-networks-for-production-with-intels-openvino-a7ee3a6883d) Cutting pre- and post-processing operations from TensorFlow graphs